In [8]:
import bs4
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests


In [11]:


#Function that open the url with selenium, and return the page source
def getPageBySel(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--remote-debugging-port=9222")
    options.add_argument("--window-size=1920x1080")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    page = driver.page_source
    driver.quit()
    return page


#Function that check for every page if its the last one
def is_last_page(soup):
    button = soup.find('button', {'aria-label': 'לעמוד הבא'})
    return 'disabled' in button.attrs


#hotelsArr that contain the first page of each city
hotelsArr = { f'https://www.booking.com/searchresults.he.html?ss=%D7%9C%D7%95%D7%A0%D7%93%D7%95%D7%9F&ssne=%D7%9C%D7%95%D7%A0%D7%93%D7%95%D7%9F&ssne_untouched=%D7%9C%D7%95%D7%A0%D7%93%D7%95%D7%9F&label=gen173nr-1BCAEoggI46AdIM1gEaGqIAQGYAQ64ARfIAQzYAQHoAQGIAgGoAgO4ArbcsaMGwAIB0gIkN2ZmYmU5ZDctYjY4Zi00ZWYxLWJjMTAtYWY0ZGQ5MzBmNmY42AIF4AIB&sid=c594a313ddbee49f67a9bb84bd30aebb&aid=304142&lang=he&sb=1&src_elem=sb&src=index&dest_id=-2601889&dest_type=city&checkin=2024-01-04&checkout=2024-01-07&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&offset='}
#creating arr for each col
hotels=[]
links=[]
prices=[]

#Loop that run on the hotels Arr
for x in hotelsArr:
    offset=25
    
# Loop that run on the first 40 pages (offset+25 each time)
    while offset < 500:
        page =x+str(offset)
        
        while True: #Loop that make sure that the page loaded successfully 
            temp=getPageBySel(page)
            soup = BeautifulSoup(temp, 'html.parser')
            if len(soup("h3",{"class":"a4225678b2"}))>0:
                break
            else:
                time.sleep(1.5)
        if is_last_page(soup):
            break
        else:
            
                
            for element in soup.select(".fcab3ed991.fbd1d3018c.e729ed5ab6"):
                # Find the price
                price = element.get_text(strip=True) if element else 'N/A'
                numeric_string = re.sub(r"[^\d.]", "", price)
                hotel_price = int(numeric_string)
                
                # Append the price to the prices list
                prices.append(hotel_price)
              
                            

                
            
                
                # Extract the hotel name and link
            for element in soup("h3", {"class": "a4225678b2"}):
                # Find the hotel link
                link = element('a')[0]['href']
                
                # Find the hotel name
                name = element.select_one('.fcab3ed991.a23c043802').get_text() if element else 'N/A'
                # Append the data to the respective lists
                
                hotels.append(name)
                links.append(link)
            offset= offset+25 #Move to the next page
            

        
        
    
# Ensure all lists have the same length
length = min(len(hotels), len(links), len(prices))
hotels = hotels[:length]
links = links[:length]
prices = prices[:length]

#Take the data into dataframe
df = pd.DataFrame({
    'Hotel':hotels, 'Link':links, 'Prices':prices
})


#Df to csv
df.to_csv('hotels_list123.csv', index=True)
print(df)





C:\Users\97254\AppData\Local\Temp\ipykernel_30176\710831071.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"C:\Users\97254\OneDrive\שולחן העבודה\Data Science - Project\chromedriver")


                                     Hotel  \
0                     Lancaster Gate Hotel   
1                   Assembly Covent Garden   
2                   Merit Kensington Hotel   
3              The Montague On The Gardens   
4                              Hotel Saint   
..                                     ...   
470                Grange Buckingham Hotel   
471  Mascutstay 2 Beds London skyline view   
472                      Moxy London Excel   
473                Veeve - Hoxton Hideaway   
474                  The Brook Green Hotel   

                                                  Link  Prices  
0    https://www.booking.com/hotel/gb/lancaster-gat...    2040  
1    https://www.booking.com/hotel/gb/assembly.he.h...    2036  
2    https://www.booking.com/hotel/gb/windsorhouseh...    2337  
3    https://www.booking.com/hotel/gb/montagueredca...    3491  
4    https://www.booking.com/hotel/gb/hotel-saint-l...    2271  
..                                                 ...   